# 09. Comparative Analysis
## Synthetic Instruction Tuner - Method Comparison

This notebook provides comprehensive comparison of different adaptation methods:
1. Load efficiency metrics from all methods
2. Load benchmark evaluation results
3. Compare performance vs efficiency trade-offs
4. Generate visualizations and summary tables
5. Export results for report

**Methods Compared**:
- LoRA (r=8, alpha=16)
- Prompt Tuning (20 virtual tokens)
- DPO (preference alignment)
- Zero-shot Baseline

## 1. Setup

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Project path
PROJECT_ROOT = "/content/drive/MyDrive/synthetic-instruction-tuner"

In [ ]:
# Load configuration
import json

with open(f"{PROJECT_ROOT}/config.json", 'r') as f:
    config = json.load(f)

print("Configuration loaded!")

In [ ]:
import json
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

METRICS_DIR = f"{PROJECT_ROOT}/evaluation/metrics"
FIGURES_DIR = f"{PROJECT_ROOT}/evaluation/figures"
os.makedirs(FIGURES_DIR, exist_ok=True)

print("Libraries loaded!")

## 2. Load Efficiency Metrics

In [ ]:
# Load metrics from all methods
def load_metrics(method_name: str) -> dict:
    """Load metrics for a specific method."""
    path = f"{METRICS_DIR}/{method_name}_metrics.json"
    if os.path.exists(path):
        with open(path, 'r') as f:
            return json.load(f)
    else:
        print(f"Warning: {path} not found")
        return None

# Load all available metrics
methods = ['lora', 'prompt_tuning', 'dpo']
all_metrics = {}

for method in methods:
    metrics = load_metrics(method)
    if metrics:
        all_metrics[method] = metrics
        print(f"Loaded metrics for: {method}")

print(f"\nTotal methods loaded: {len(all_metrics)}")

In [ ]:
# Create comparison DataFrame
if all_metrics:
    df_metrics = pd.DataFrame(all_metrics).T
    df_metrics.index.name = 'Method'
    
    print("=== Efficiency Metrics Comparison ===")
    display(df_metrics)
else:
    print("No metrics available. Run training notebooks first.")
    # Create sample data for demonstration
    sample_data = {
        'lora': {
            'method': 'lora_r8',
            'trainable_params': 4194304,
            'total_params': 3000000000,
            'trainable_ratio_percent': 0.14,
            'peak_memory_gb': 8.5,
            'training_time_hours': 6.5,
            'inference_tokens_per_sec': 25.0,
            'train_loss': 1.45,
            'eval_loss': 1.52
        },
        'prompt_tuning': {
            'method': 'prompt_tuning',
            'trainable_params': 81920,
            'total_params': 3000000000,
            'trainable_ratio_percent': 0.003,
            'peak_memory_gb': 6.2,
            'training_time_hours': 3.5,
            'inference_tokens_per_sec': 28.0,
            'train_loss': 1.85,
            'eval_loss': 1.92
        },
        'dpo': {
            'method': 'dpo',
            'trainable_params': 4194304,
            'total_params': 3000000000,
            'trainable_ratio_percent': 0.14,
            'peak_memory_gb': 12.5,
            'training_time_hours': 4.0,
            'inference_tokens_per_sec': 25.0,
            'train_loss': 0.65,
            'eval_loss': 0.72
        }
    }
    all_metrics = sample_data
    df_metrics = pd.DataFrame(sample_data).T
    print("Using sample data for demonstration:")
    display(df_metrics)

## 3. Load Benchmark Results

In [ ]:
# Load benchmark results if available
benchmark_path = f"{METRICS_DIR}/benchmark_results.json"

if os.path.exists(benchmark_path):
    with open(benchmark_path, 'r') as f:
        benchmark_results = json.load(f)
    print("Benchmark results loaded!")
else:
    # Sample benchmark data
    benchmark_results = {
        'zero_shot': {
            'mmlu': 42.5,
            'hellaswag': 55.2,
            'arc_easy': 58.3,
            'truthfulqa': 35.1
        },
        'lora': {
            'mmlu': 48.2,
            'hellaswag': 62.5,
            'arc_easy': 65.8,
            'truthfulqa': 42.3
        },
        'prompt_tuning': {
            'mmlu': 45.1,
            'hellaswag': 58.8,
            'arc_easy': 61.2,
            'truthfulqa': 38.5
        },
        'dpo': {
            'mmlu': 49.5,
            'hellaswag': 64.2,
            'arc_easy': 67.5,
            'truthfulqa': 48.9
        }
    }
    print("Using sample benchmark data for demonstration")

df_benchmarks = pd.DataFrame(benchmark_results).T
df_benchmarks.index.name = 'Method'

print("\n=== Benchmark Results ===")
display(df_benchmarks)

## 4. Efficiency Visualizations

In [ ]:
# Create comprehensive comparison plots
fig, axes = plt.subplots(2, 3, figsize=(16, 10))

methods_list = list(all_metrics.keys())
colors = sns.color_palette("husl", len(methods_list))

# 1. Trainable Parameters
ax = axes[0, 0]
params = [all_metrics[m]['trainable_params'] for m in methods_list]
bars = ax.bar(methods_list, params, color=colors)
ax.set_ylabel('Parameters')
ax.set_title('Trainable Parameters')
ax.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
for bar, val in zip(bars, params):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
            f'{val:,.0f}', ha='center', va='bottom', fontsize=8)

# 2. Trainable Ratio
ax = axes[0, 1]
ratios = [all_metrics[m]['trainable_ratio_percent'] for m in methods_list]
bars = ax.bar(methods_list, ratios, color=colors)
ax.set_ylabel('Percentage (%)')
ax.set_title('Trainable Parameter Ratio')
for bar, val in zip(bars, ratios):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
            f'{val:.3f}%', ha='center', va='bottom', fontsize=9)

# 3. Peak Memory
ax = axes[0, 2]
memory = [all_metrics[m]['peak_memory_gb'] for m in methods_list]
bars = ax.bar(methods_list, memory, color=colors)
ax.set_ylabel('Memory (GB)')
ax.set_title('Peak GPU Memory Usage')
ax.axhline(y=15, color='r', linestyle='--', label='T4 Limit (15GB)')
ax.legend()
for bar, val in zip(bars, memory):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
            f'{val:.1f}', ha='center', va='bottom', fontsize=9)

# 4. Training Time
ax = axes[1, 0]
time_hrs = [all_metrics[m]['training_time_hours'] for m in methods_list]
bars = ax.bar(methods_list, time_hrs, color=colors)
ax.set_ylabel('Hours')
ax.set_title('Training Time')
for bar, val in zip(bars, time_hrs):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
            f'{val:.1f}h', ha='center', va='bottom', fontsize=9)

# 5. Inference Speed
ax = axes[1, 1]
speed = [all_metrics[m]['inference_tokens_per_sec'] for m in methods_list]
bars = ax.bar(methods_list, speed, color=colors)
ax.set_ylabel('Tokens/sec')
ax.set_title('Inference Speed')
for bar, val in zip(bars, speed):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
            f'{val:.1f}', ha='center', va='bottom', fontsize=9)

# 6. Eval Loss
ax = axes[1, 2]
losses = [all_metrics[m].get('eval_loss', 0) for m in methods_list]
bars = ax.bar(methods_list, losses, color=colors)
ax.set_ylabel('Loss')
ax.set_title('Evaluation Loss')
for bar, val in zip(bars, losses):
    if val:
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
                f'{val:.2f}', ha='center', va='bottom', fontsize=9)

plt.suptitle('Adaptation Method Efficiency Comparison', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(f"{FIGURES_DIR}/efficiency_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved to {FIGURES_DIR}/efficiency_comparison.png")

## 5. Benchmark Performance Comparison

In [ ]:
# Benchmark comparison radar chart
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Bar chart comparison
ax = axes[0]
df_benchmarks.plot(kind='bar', ax=ax, width=0.8)
ax.set_xlabel('Method')
ax.set_ylabel('Accuracy (%)')
ax.set_title('Benchmark Performance by Method')
ax.legend(title='Benchmark', bbox_to_anchor=(1.02, 1), loc='upper left')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Average performance
ax = axes[1]
avg_scores = df_benchmarks.mean(axis=1)
bars = ax.bar(avg_scores.index, avg_scores.values, color=colors[:len(avg_scores)])
ax.set_ylabel('Average Accuracy (%)')
ax.set_title('Average Benchmark Performance')
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
for bar, val in zip(bars, avg_scores.values):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height(), 
            f'{val:.1f}%', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.savefig(f"{FIGURES_DIR}/benchmark_comparison.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved to {FIGURES_DIR}/benchmark_comparison.png")

## 6. Performance vs Efficiency Trade-off

In [ ]:
# Create trade-off analysis
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Scatter: Performance vs Training Time
ax = axes[0]
for i, method in enumerate(methods_list):
    if method in df_benchmarks.index:
        x = all_metrics[method]['training_time_hours']
        y = df_benchmarks.loc[method].mean()
        ax.scatter(x, y, s=200, c=[colors[i]], label=method, edgecolors='black', linewidth=2)
        ax.annotate(method, (x, y), textcoords="offset points", 
                    xytext=(0, 10), ha='center', fontsize=10)

ax.set_xlabel('Training Time (hours)')
ax.set_ylabel('Average Benchmark Score (%)')
ax.set_title('Performance vs Training Time')
ax.grid(True, alpha=0.3)

# Scatter: Performance vs Memory
ax = axes[1]
for i, method in enumerate(methods_list):
    if method in df_benchmarks.index:
        x = all_metrics[method]['peak_memory_gb']
        y = df_benchmarks.loc[method].mean()
        ax.scatter(x, y, s=200, c=[colors[i]], label=method, edgecolors='black', linewidth=2)
        ax.annotate(method, (x, y), textcoords="offset points", 
                    xytext=(0, 10), ha='center', fontsize=10)

ax.set_xlabel('Peak Memory (GB)')
ax.set_ylabel('Average Benchmark Score (%)')
ax.set_title('Performance vs Memory Usage')
ax.axvline(x=15, color='r', linestyle='--', label='T4 Limit', alpha=0.7)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f"{FIGURES_DIR}/tradeoff_analysis.png", dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved to {FIGURES_DIR}/tradeoff_analysis.png")

## 7. Summary Table for Report

In [ ]:
# Create comprehensive summary table
summary_data = []

for method in methods_list:
    row = {
        'Method': method.upper(),
        'Trainable Params': f"{all_metrics[method]['trainable_params']:,}",
        'Param Ratio (%)': f"{all_metrics[method]['trainable_ratio_percent']:.3f}",
        'Memory (GB)': f"{all_metrics[method]['peak_memory_gb']:.1f}",
        'Train Time (h)': f"{all_metrics[method]['training_time_hours']:.1f}",
        'Inference (tok/s)': f"{all_metrics[method]['inference_tokens_per_sec']:.1f}",
    }
    
    # Add benchmark scores if available
    if method in df_benchmarks.index:
        row['Avg Benchmark'] = f"{df_benchmarks.loc[method].mean():.1f}%"
    else:
        row['Avg Benchmark'] = 'N/A'
    
    summary_data.append(row)

# Add zero-shot baseline
if 'zero_shot' in df_benchmarks.index:
    summary_data.insert(0, {
        'Method': 'ZERO-SHOT',
        'Trainable Params': '0',
        'Param Ratio (%)': '0.000',
        'Memory (GB)': 'N/A',
        'Train Time (h)': '0.0',
        'Inference (tok/s)': 'N/A',
        'Avg Benchmark': f"{df_benchmarks.loc['zero_shot'].mean():.1f}%"
    })

df_summary = pd.DataFrame(summary_data)

print("=" * 80)
print("COMPREHENSIVE COMPARISON SUMMARY")
print("=" * 80)
display(df_summary)

# Save to CSV
df_summary.to_csv(f"{METRICS_DIR}/comparison_summary.csv", index=False)
print(f"\nSaved to {METRICS_DIR}/comparison_summary.csv")

## 8. Key Findings

In [ ]:
# Generate key findings
print("=" * 60)
print("KEY FINDINGS")
print("=" * 60)

# Find best methods for each metric
findings = []

# Best performance
if 'dpo' in df_benchmarks.index:
    best_perf = df_benchmarks.mean(axis=1).idxmax()
    findings.append(f"1. Best Performance: {best_perf.upper()} ({df_benchmarks.loc[best_perf].mean():.1f}% avg)")

# Most efficient (params)
min_params_method = min(all_metrics.keys(), key=lambda m: all_metrics[m]['trainable_params'])
findings.append(f"2. Most Parameter-Efficient: {min_params_method.upper()} ({all_metrics[min_params_method]['trainable_params']:,} params)")

# Fastest training
fastest_method = min(all_metrics.keys(), key=lambda m: all_metrics[m]['training_time_hours'])
findings.append(f"3. Fastest Training: {fastest_method.upper()} ({all_metrics[fastest_method]['training_time_hours']:.1f} hours)")

# Lowest memory
min_mem_method = min(all_metrics.keys(), key=lambda m: all_metrics[m]['peak_memory_gb'])
findings.append(f"4. Lowest Memory: {min_mem_method.upper()} ({all_metrics[min_mem_method]['peak_memory_gb']:.1f} GB)")

for finding in findings:
    print(finding)

print("\n" + "=" * 60)
print("RECOMMENDATIONS")
print("=" * 60)
print("""
- For best quality: Use LoRA + DPO pipeline
- For limited resources: Use Prompt Tuning (minimal parameters)
- For quick experiments: Start with Prompt Tuning, then upgrade to LoRA
- For production: LoRA provides best quality/efficiency balance
""")

## 9. Export for Report

In [ ]:
# Export all results
report_data = {
    "efficiency_metrics": all_metrics,
    "benchmark_results": benchmark_results,
    "key_findings": findings,
    "summary_table": summary_data
}

with open(f"{METRICS_DIR}/full_comparison_report.json", 'w') as f:
    json.dump(report_data, f, indent=2)

print(f"Full report saved to {METRICS_DIR}/full_comparison_report.json")

# List all generated files
print("\n=== Generated Files ===")
for f in os.listdir(FIGURES_DIR):
    print(f"  - {FIGURES_DIR}/{f}")
for f in os.listdir(METRICS_DIR):
    if f.endswith('.json') or f.endswith('.csv'):
        print(f"  - {METRICS_DIR}/{f}")

## ✓ Comparative Analysis Complete!

### Generated Outputs:
1. **Efficiency Comparison Chart**: `evaluation/figures/efficiency_comparison.png`
2. **Benchmark Comparison Chart**: `evaluation/figures/benchmark_comparison.png`
3. **Trade-off Analysis**: `evaluation/figures/tradeoff_analysis.png`
4. **Summary CSV**: `evaluation/metrics/comparison_summary.csv`
5. **Full Report JSON**: `evaluation/metrics/full_comparison_report.json`

### Key Metrics Compared:
- Trainable Parameters
- Memory Usage
- Training Time
- Inference Speed
- Benchmark Performance

### Next Steps:
1. Use these visualizations in your Written Report
2. Include key findings in your Presentation
3. Discuss trade-offs in Comparative Analysis section